In [ ]:
using Pkg
Pkg.activate("..")

using Neurthino
using BenchmarkTools
using Profile
using ProfileVega
using StaticArrays

In [ ]:
const osc = OscillationParameters(3)

@code_warntype Hamiltonian(osc)

In [ ]:
function foo()
    osc = OscillationParameters(3);

    U = PMNSMatrix(osc)
    H = Hamiltonian(osc)
end

@code_warntype foo()

In [ ]:
function main()
    osc = OscillationParameters(3);
    mixingangle!(osc, 1=>2, 0.594);
    mixingangle!(osc, 1=>3, 0.161);
    mixingangle!(osc, 2=>3, 0.700);
    cpphase!(osc, 1=>3, 5.34);
    masssquareddiff!(osc, 2=>3, -2.32e-3);
    masssquareddiff!(osc, 1=>3, -2.32e-3-7.59e-5);
    masssquareddiff!(osc, 1=>2, -7.59e-5);

    U = PMNSMatrix(osc)
    H = Hamiltonian(osc)

    samples = 100
    
    coszenith = range(-1; stop=0, length=samples)
    zenith = acos.(coszenith)
    energies = 10 .^(range(0;stop=2,length=samples))

    doosc(osc, zenith, energies)
end



function doosc(osc, zenith, energies)
    data_points = zeros((3,3,length(zenith), length(energies)))

    first_call = true
    for (i,z) in enumerate(zenith)
        sec, dens = Neurthino.prempath(z, 2.5, samples=50)
        if first_call
            Neurthino.mattertransprob(osc, energies, dens, sec)
            first_call = false
        end
        P = @profile Neurthino.mattertransprob(osc, energies, dens, sec)
        for (k,l) in Iterators.product(fill(1:3, 2)...)
            @inbounds data_points[k,l,i,:] = map(x->x[k,l], P)
        end
    end
    return data_points
end

In [ ]:
ProfileVega.view()

In [ ]:
Profile.print()